In [1]:

NUM_PROC_MAP = 100 # expect 30 min with single process

In [2]:
import os
import os
os.environ['HF_HOME'] = '/data/loriss21dm/cache'
import datasets
# datasets.set_caching_enabled(False)
from datasets import load_dataset
import datasets
import torch
import json

As in the official [preprocessing repo](https://github.com/babylm/babylm_data_preprocessing), we get the number of words via `line.split()`:

In [3]:

# from nltk.tokenize import RegexpTokenizer
# tokenizer = RegexpTokenizer(r'\w+')
# word_count = lambda d: len([w for w in tokenizer.tokenize(d) if w.isalnum()])
# sum([word_count(d) for d in dataset["text"]])
word_count = lambda d: len(d.split())
word_count_dataset = lambda dataset:sum([word_count(d) for d in dataset["text"]])

In [4]:
!mkdir curricula

mkdir: cannot create directory ‘curricula’: File exists


In [5]:
import os.path


### Curriculum 2023 dataset

The exact data from Thoma et al 2023. Note that the model does not see all examples in each epoch.

In [6]:
# from datasets import load_dataset
# import os
# os.environ['HF_HOME'] = '/data/loriss21dm/cache'
# # dev = load_dataset("babylm-anon/dev-data")

In [7]:
# LAST_GEN_DIR = "last_gen"
# EPOCHS_PER_STAGE = 10
# datasets_stages = []
# orders_stages = []

# torch.manual_seed(0)
# for folder in (sorted(os.listdir(LAST_GEN_DIR))):
#     d = load_dataset("text", data_dir =os.path.join(LAST_GEN_DIR, folder), cache_dir='/data/loriss21dm/cache')["train"] 
#     d = d.shuffle(seed=42) # we shuffle within the stage
#     d = datasets.Dataset.from_dict(d[0:len(d)//10])
#     datasets_stages.append(d)
#     offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
#     for _ in range(0,EPOCHS_PER_STAGE):
#         orders_stages.append(torch.arange(offset,len(d)+offset))
    


# dataset = datasets.concatenate_datasets(datasets_stages)

# dataset.save_to_disk("./curricula/datasets/curriculum_100M_2023")
# #assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

# torch.save(orders_stages, "./curricula/curriculum_100M_2023")
# len(orders_stages)

### Curriculum with 10M 2024 dataset

In [8]:
args = {
    "dataset_folder": "./train_10M",
   
}
args["raw_dataset_folder_babylm"] = "./train_10M"
args["raw_eval_dataset_folder_babylm"] = "./train_100M"
args["dataset_folder"] = "./curricula/datasets/curriculum_10M_2024"
args["curriculum_path"] = "./curricula/curriculum_10M_2024"
args["eval_dataset_folder"] = "./curricula/datasets/curriculum_10M_2024_eval"
args["epochs_per_stage"] = 2

args["curriculum"] = {
    "C1: Child Directed Speech": ["childes.train"],
    "C2: Unscripted Dialogue": ["switchboard.train","bnc_spoken.train"],
    "C3: Scripted Dialogue": ["open_subtitles.train", ],
    "C4: Wiki": [ "simple_wiki.train"],
    "C5: Written English": ["gutenberg.train"],

}

In [9]:
def add_source(entry, source):
    entry["source"] = source
    return entry
def create_dataset(curriculum, raw_dataset_folder):
    datasets_stages = []
    orders_stages = []
    for stage, files in curriculum.items():
        d = datasets.concatenate_datasets(
                [
                    load_dataset("text", data_files =os.path.join(raw_dataset_folder, file),download_mode="force_redownload")["train"] 
                    .map(lambda entry: add_source(entry, file),  num_proc=NUM_PROC_MAP)
                    for file in files
                ]
            )
        d = d.shuffle(seed=42) # we shuffle with the stage
        datasets_stages.append((d,stage))
        offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
        for i in range(0,args["epochs_per_stage"]):
            indices = torch.arange(offset,len(d)+offset) # and then shuffle again (across epochs within stages)
            orders_stages.append(indices[torch.randperm(len(indices))])
    return datasets_stages, orders_stages
        

if True or not os.path.exists(args["dataset_folder"]):
    # pretraining data
    torch.manual_seed(0)
    datasets_stages, orders_stages = create_dataset(args["curriculum"], args["raw_dataset_folder_babylm"])
    dataset = datasets.concatenate_datasets([d for d,_ in datasets_stages])
    dataset = dataset.shuffle(seed=42)
    dataset.save_to_disk(args["dataset_folder"],num_proc=NUM_PROC_MAP)
    torch.save(orders_stages, args["curriculum_path"])

    # eval data is a split of (100M dataset - 10M dataset)
    # of size 0.05*len(10M dataset)
    torch.manual_seed(0)
    eval_datasets_stages, _ = create_dataset(args["curriculum"], args["raw_eval_dataset_folder_babylm"])



    # remove all strings that are in the eval dataset 
    ## basically (100M dataset - 10M dataset) but one source at a time as that is faster
    datasets_eval = []
    lookup = {stage: d for d, stage in datasets_stages}
    # print(lookup)
    for e, stage in eval_datasets_stages:
        if stage in lookup.keys():
            l = list(set(e["text"]) - set(lookup[stage]["text"]))
            datasets_eval.append(datasets.Dataset.from_dict({"text": l, "stage": [stage]*len(l)}))
        else:
            print("warning: adding all of ",  stage )
            datasets_eval.append(e)
    dataset_eval = datasets.concatenate_datasets(datasets_eval)

    # do a stratified split, requires casting to class
    def copy_source_col(x):
        x["stage_"] = x["stage"]
        return x
    dataset_eval = dataset_eval.map(copy_source_col,num_proc=NUM_PROC_MAP)
    dataset_eval = dataset_eval.class_encode_column("stage_")
    dataset_eval = dataset_eval.train_test_split(test_size=int(len(dataset)*0.05), seed=42, stratify_by_column="stage_",keep_in_memory=True)["test"]
    dataset_eval = dataset_eval.remove_columns("stage_")

    print(len(dataset), len(dataset_eval))
    dataset_eval = dataset_eval.shuffle(seed=42)
    
    # save
    datasets.Dataset.from_dict(dataset_eval.to_dict()).save_to_disk(args["eval_dataset_folder"], num_proc=NUM_PROC_MAP ) # converting to dict and back to speed up substantially (or substantial slowdown caused by conversions above)



    # assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))



    len_full_dataset = len(dataset)
    args["epoch_equivalents"] = sum([len(o) for o in orders_stages]) / len_full_dataset
    args["epochs"] = len(orders_stages)




    with open("./configs/curriculum_10M_2024", 'w') as f:
        json.dump(args, f)
args


Generating train split:   0%|          | 0/580000 [00:00<?, ? examples/s]

Map (num_proc=100):   0%|          | 0/580000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map (num_proc=100):   0%|          | 0/18000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/90000 [00:00<?, ? examples/s]

...

Map (num_proc=100):   0%|          | 0/6502845 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/6502845 [00:00<?, ? examples/s]

1179014 58950


Saving the dataset (0/100 shards):   0%|          | 0/58950 [00:00<?, ? examples/s]

{'dataset_folder': './curricula/datasets/curriculum_10M_2024',
 'raw_dataset_folder_babylm': './train_10M',
 'raw_eval_dataset_folder_babylm': './train_100M',
 'curriculum_path': './curricula/curriculum_10M_2024',
 'eval_dataset_folder': './curricula/datasets/curriculum_10M_2024_eval',
 'epochs_per_stage': 2,
 'curriculum': {'C1: Child Directed Speech': ['childes.train'],
  'C2: Unscripted Dialogue': ['switchboard.train', 'bnc_spoken.train'],
  'C3: Scripted Dialogue': ['open_subtitles.train'],
  'C4: Wiki': ['simple_wiki.train'],
  'C5: Written English': ['gutenberg.train']},
 'epoch_equivalents': 2.0,
 'epochs': 10}

In [12]:
import pandas as pd
import datasets
dataset = datasets.load_from_disk(args["dataset_folder"])
df = dataset.to_pandas()
df["word_count"] = df["text"].apply(word_count)
df["stage"] = df["source"].apply(lambda source: next((name for name, files in args["curriculum"].items() if source in files), None))
display(df.value_counts(["source"], normalize=True))
display(df.groupby(by=["source"])["word_count"].sum() / df["word_count"].sum())
display(df.groupby(by=["stage"])["word_count"].sum() / df["word_count"].sum())
display(df.groupby(by=["stage"])["word_count"].count() / len(dataset))

Loading dataset from disk:   0%|          | 0/100 [00:00<?, ?it/s]

source              
childes.train           0.491936
open_subtitles.train    0.305340
bnc_spoken.train        0.076335
gutenberg.train         0.055991
simple_wiki.train       0.055131
switchboard.train       0.015267
Name: proportion, dtype: float64

source
bnc_spoken.train        0.093683
childes.train           0.285278
gutenberg.train         0.255128
open_subtitles.train    0.205135
simple_wiki.train       0.146029
switchboard.train       0.014747
Name: word_count, dtype: float64

stage
C1: Child Directed Speech    0.285278
C2: Unscripted Dialogue      0.108430
C3: Scripted Dialogue        0.205135
C4: Wiki                     0.146029
C5: Written English          0.255128
Name: word_count, dtype: float64

stage
C1: Child Directed Speech    0.491936
C2: Unscripted Dialogue      0.091602
C3: Scripted Dialogue        0.305340
C4: Wiki                     0.055131
C5: Written English          0.055991
Name: word_count, dtype: float64

### Stratified 10M Curriculum 

This creates 5 stages of equal size totaling 10M tokens from the raw datasets

In [ ]:
!mkdir babylm_data_preprocessing/tmp
!git clone https://github.com/babylm/babylm_data_preprocessing.git
!git clone https://github.com/pgcorpus/gutenberg.git babylm_data_preprocessing/tmp/gutenberg


First, download the raw datasets as described in the official [preprocessing repo](https://github.com/babylm/babylm_data_preprocessing)). Note that the link to `simplewiki` expired, so we use a more recent dump.

In [ ]:


# !mkdir babylm_data_preprocessing/preprocessed_data

# !curl https://raw.githubusercontent.com/phueb/BabyBERTa/master/data/corpora/aochildes.txt > babylm_data_preprocessing/preprocessed_data/aochildes.txt
# !curl https://raw.githubusercontent.com/NathanDuran/Switchboard-Corpus/master/swda_data/full_set.txt > babylm_data_preprocessing/preprocessed_data/switchboard.txt
# !curl https://raw.githubusercontent.com/NathanDuran/Switchboard-Corpus/master/swda_data/full_set.txt > babylm_data_preprocessing/preprocessed_data/switchboard.txt

# !curl http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz > babylm_data_preprocessing/preprocessed_data/CBTest.tgz
# !tar -xvzf babylm_data_preprocessing/preprocessed_data/CBTest.tgz -C babylm_data_preprocessing/preprocessed_data
# !mv babylm_data_preprocessing/preprocessed_data/CBTest/data/cbt_*  babylm_data_preprocessing/preprocessed_data/
# !rm -rf babylm_data_preprocessing/preprocessed_data/CBTest babylm_data_preprocessing/preprocessed_data/CBTest.tgz

# !gdown 1nbUCWCAvtqI1-WQxzmyqQmddgsZtzdpR
# !unzip -o children_stories.txt.zip -d babylm_data_preprocessing/preprocessed_data
# !rm children_stories.txt.zip

# !gdown 1vW0o7K6Gj_IYTzriWEjmCnrylCWb8DbY
# !unzip -o open_subtitles.txt.zip -d babylm_data_preprocessing/preprocessed_data
# !rm open_subtitles.txt.zip

# !gdown 19GipY95MW3LrfO_kArmIC0KYy7mfCb1l
# !unzip -o wikipedia.txt.zip -d babylm_data_preprocessing/preprocessed_data
# !rm wikipedia.txt.zip
# !gdown 1R2xWtNeVX48RiFA7vErL1pNtws3XEsYP
# !unzip -o qed.zip -d babylm_data_preprocessing/tmp
# !rm qed.zip

# %run babylm_data_preprocessing/preprocess_qed.py babylm_data_preprocessing/tmp/en babylm_data_preprocessing/tmp/qed
# # !cat babylm_data_preprocessing/tmp/qed/* >> babylm_data_preprocessing/preprocessed_data/qed.txt
# !find babylm_data_preprocessing/tmp/qed/ -type f -exec cat {} + > babylm_data_preprocessing/preprocessed_data/qed.txt
# !rm -rf babylm_data_preprocessing/tmp/qed babylm_data_preprocessing/tmp/en 
# # simplewiki
# !curl https://dumps.wikimedia.org/simplewiki/20241101/simplewiki-20241101-pages-articles.xml.bz2 > babylm_data_preprocessing/tmp/wiki.bz2 # TODO backup
# !bzip2 -d babylm_data_preprocessing/tmp/wiki.bz2
# !python -m wikiextractor.WikiExtractor babylm_data_preprocessing/tmp/wiki -o babylm_data_preprocessing/tmp/wiki_txt

# # https://github.com/babylm/babylm_data_preprocessing/blob/main/preprocess_simple_wiki.py
# # have to change working dir
# import os
# import re

# out_file = open(os.path.join("babylm_data_preprocessing", "preprocessed_data/simple_wiki.txt"), "w")
# wiki_dir = os.path.join("babylm_data_preprocessing","tmp", "wiki_txt")
# for d1 in os.listdir(wiki_dir):
# 	for f in os.listdir(os.path.join(wiki_dir, d1)):
# 		with open(os.path.join(wiki_dir, d1, f)) as input:
# 			title = None
# 			doc = []
# 			for line in input:
# 				if line.startswith("<doc"):
# 					line = next(input)
# 					title = line
# 				elif re.match(r"^\s*$", line):
# 					continue
# 				elif "</doc>" in line:
# 					if len(doc) > 0:
# 						out_file.write(title)
# 						out_file.write("".join(doc))
# 						out_file.write("\n")
# 						doc = []
# 				else:
# 					doc.append(line)
# !rm -rf babylm_data_preprocessing/tmp/wiki babylm_data_preprocessing/tmp/wiki_txt					

# !mkdir babylm_data_preprocessing/tmp/bnc_spoken
# !curl https://llds.ling-phil.ox.ac.uk/llds/xmlui/bitstream/handle/20.500.14106/2554/2554.zip > babylm_data_preprocessing/tmp/bnc_spoken/bnc.zip
# !unzip -q babylm_data_preprocessing/tmp/bnc_spoken/bnc.zip -d babylm_data_preprocessing/tmp/bnc_spoken/
# !(for z in babylm_data_preprocessing/tmp/bnc_spoken/download/Texts/*; do for y in $z/*; do for x in $y/*; do sed '2q;d' $x | grep "^<stext" -q && cp $x babylm_data_preprocessing/tmp/bnc_spoken/; done; done; done)
# %run babylm_data_preprocessing/preprocess_bnc.py babylm_data_preprocessing/tmp/bnc_spoken/ babylm_data_preprocessing/preprocessed_data/bnc_spoken.txt
# !rm -rf babylm_data_preprocessing/tmp/bnc_spoken


# the get_data.py script ignores the `metadata` param
%cd babylm_data_preprocessing/tmp/gutenberg 
!mkdir metadata

# this can take a day or two...
%run get_data.py 

%cd babylm_data_preprocessing/tmp/gutenberg 
!mkdir metadata
# the repo contains a tokenizer but for an unspecified version of nltk
# we download a current one
# see https://github.com/pgcorpus/gutenberg/issues/5
import nltk
nltk.data.path=["src/nltk_data"]
nltk.download('punkt_tab',download_dir='src/nltk_data')
%run process_data.py
%cd ../../..


In [ ]:
import pandas as pd
df = pd.read_csv("metadata/metadata.csv").set_index("id")

In [ ]:
df[df["language"]== "['en']"]

In [ ]:
# from the babylm preprocessing repo: https://github.com/babylm/babylm_data_preprocessing/blob/main/get_gutenberg_modern_en.py
import pandas as pd
import shutil
import os

df = pd.read_csv("babylm_data_preprocessing/tmp/gutenberg/metadata/metadata.csv")
df_modern_en = df[(df["language"] == "['en']") & (df["authoryearofbirth"] > 1850)]
modern_en_ids = set(df_modern_en["id"])

os.makedirs("babylm_data_preprocessing/tmp/gutenberg_modern_en")
for f in os.listdir("babylm_data_preprocessing/tmp/gutenberg/data/text"): 
    if f.split("_")[0] in modern_en_ids:
        shutil.copyfile("babylm_data_preprocessing/tmp/gutenberg/data/text/" + f, "babylm_data_preprocessing/tmp/gutenberg_modern_en/" + f)
!find babylm_data_preprocessing/tmp/gutenberg_modern_en/ -type f -exec cat {} + > babylm_data_preprocessing/preprocessed_data/gutenberg.txt
# you may delete the tmp/gutenberg dir now 
# # !rm -rf babylm_data_preprocessing/tmp/gutenberg
# # !rm -rf metadata 

In [ ]:
args["dataset_folder"] = "./curricula/datasets/curriculum_stratified_10M_2024"
args["curriculum_path"] = "./curricula/curriculum_stratified_10M_2024"
args["eval_dataset_folder"] = "./curricula/datasets/curriculum_stratified_10M_2024_eval"
args["epochs_per_stage"] = 10
args["raw_dataset_folder"] = "./babylm_data_preprocessing/preprocessed_data"
args["curriculum"] = {
    "C1: Child Directed Speech" : ["aochildes.txt", ],
    "C2: Children's Books": ["children_stories.txt", "cbt_test.txt", "cbt_train.txt","cbt_valid.txt"],
    "C3: Dialogue": ["switchboard.txt", "bnc_spoken.txt", "open_subtitles.txt"],
    "C4: Educational": ["qed.txt", "simple_wiki.txt"],
    "C5: Written English": ["wikipedia.txt",]# "gutenberg.txt"]

} 

In [ ]:
10_000_000*0.05

In [ ]:

datasets_stages = []
datasets_stages_eval = []
orders_stages = []

BUDGET_PER_STAGE = 10_000_000 // len(args["curriculum"])
SIZE_EVAL_SPLIT = 0.05

def get_train_test_splits_for_stage(d):
    d = d.shuffle(seed=42)
    words_in_train_split = 0
    i = 0
    wc = word_count(d[i]["text"])
    while (BUDGET_PER_STAGE >= (words_in_train_split + wc)) and (i < len(d)):
        words_in_train_split += wc
        i+=1
        wc = word_count(d[i]["text"])

    words_in_eval_split = 0
    j = i+1
    wc = word_count(d[j]["text"])
    while ((BUDGET_PER_STAGE*SIZE_EVAL_SPLIT) >= (words_in_eval_split + wc)) and (j < len(d)):
        words_in_eval_split += wc
        j+=1
        wc = word_count(d[j]["text"])

    #      pretrain                               eval
    return datasets.Dataset.from_dict(d[0:i]), datasets.Dataset.from_dict(d[i+1:j+1]) 

if not os.path.exists(args["dataset_folder"]):
    torch.manual_seed(0)
    chunks = { stage: datasets.concatenate_datasets(
                [
                    load_dataset("text", data_files =os.path.join(args["raw_dataset_folder"], file),download_mode="force_redownload")["train"] 
                    .map(lambda entry: add_source(entry, file), num_proc=NUM_PROC_MAP)
                    for file in files
                ]
            ) for stage, files in args["curriculum"].items()}
    # print("Calculating counts")
    # counts = {name: word_count_dataset(d) for name,d in chunks.items()}
    # print(counts)
    for name,d in chunks.items():
        print("Processing", name)
        d, d_eval = get_train_test_splits_for_stage(d)
        datasets_stages.append(d)
        datasets_stages_eval.append(d_eval)
        offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
        for i in range(0,args["epochs_per_stage"]):
            indices = torch.arange(offset,len(d)+offset) # and then shuffle again (across epochs within stages)
            orders_stages.append(indices[torch.randperm(len(indices))])
        

            
    # pretraining data
    dataset = datasets.concatenate_datasets(datasets_stages)
    dataset = dataset.shuffle(seed=42)
    dataset.save_to_disk(args["dataset_folder"],num_proc=NUM_PROC_MAP)
    torch.save(orders_stages, args["curriculum_path"])


    torch.manual_seed(0)

    dataset_eval = datasets.concatenate_datasets(datasets_stages_eval)
    dataset_eval.save_to_disk(args["eval_dataset_folder"],num_proc=NUM_PROC_MAP)
    len_full_dataset = len(dataset)
    args["epoch_equivalents"] = sum([len(o) for o in orders_stages]) / len_full_dataset
    args["epochs"] = len(orders_stages)

    with open("./configs/curriculum_stratified_10M_2024", 'w') as f:
        json.dump(args, f)
    args



In [ ]:
word_count_dataset(dataset)

In [ ]:
# assert word_count_dataset(dataset) <= 10_000_000

In [ ]:
dataset = datasets.load_from_disk(args["dataset_folder"])

In [ ]:
import pandas as pd
dataset = datasets.load_from_disk(args["dataset_folder"])
df = dataset.to_pandas()
df["word_count"] = df["text"].apply(word_count)
df["stage"] = df["source"].apply(lambda source: next((name for name, files in args["curriculum"].items() if source in files), None))
display(df.value_counts(["source"]))
display(df.groupby(by=["source"])["word_count"].sum() / df["word_count"].sum())
display(df.groupby(by=["stage"])["word_count"].sum() / df["word_count"].sum())

In [ ]:
df.value_counts(["source"])

In [ ]:
df.groupby(by=["source"])["word_count"].sum() / df["word_count"].sum()

In [ ]:
df.groupby(by=["stage"])["word_count"].sum() / df["word_count"].sum()

In [ ]:
df

In [ ]:
print(counts)

In [ ]:
min(counts.values()) > BUDGET_PER_STAGE

In [ ]:
(5*BUDGET_PER_STAGE)*0.05asdf32wedscx


In [ ]:
%run process_data.py

In [ ]:
min(counts.values())*len(args["curriculum"])

In [ ]:
len(args["curriculum"])

### Random Shuffle

This mimicks the default behaviour of Huggingface's `Trainer`: Randomly shuffle dataset after each epoch.
All data is shown at each epoch.

In [ ]:
args = {
    "dataset_folder": "./curricula/datasets/curriculum_10M_2024",# reuse one created above
   
}

In [ ]:
args["raw_dataset_folder_babylm"] = "./train_10M"
args["curriculum_path"] = "./curricula/random_10M_2024"
args["eval_dataset_folder"] = "./curricula/datasets/curriculum_10M_2024_eval"
args["epochs"] = 10

In [ ]:
from datasets import load_dataset
import datasets
dataset = datasets.load_from_disk(args["dataset_folder"],num_proc=NUM_PROC_MAP,download_mode="force_redownload") # reuse one created above

In [ ]:
assert torch.equal(torch.sort(torch.randperm(len(dataset))).values, torch.arange(0,len(dataset)))

In [ ]:
args["epochs"] = 10
import torch 
torch.manual_seed(0)
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, "./curricula/random_10M_2024")
with open("./configs/random_10M_2024", 'w') as f:
    json.dump(args, f)
args

|     Groups     |Version|Filter|n-shot|Metric|Value |   |Stderr|
|----------------|-------|------|-----:|------|-----:|---|-----:|
|blimp_supplement|N/A    |none  |     0|acc   |0.4401|±  |0.0069|
|blimp_filtered  |N/A    |none  |     0|acc   |0.4579|±  |0.0019|


### Random Shuffle with a proportion of the documents beeing random tokens as a control

In [ ]:
args = {
    "dataset_folder": "./curricula/datasets/curriculum_10M_2024_noisy",# reuse one created above
   
}
args["raw_dataset_folder_babylm"] = "./train_10M"
args["curriculum_path"] = "./curricula/random_10M_2024_noisy"
args["eval_dataset_folder"] = "./curricula/datasets/curriculum_10M_2024_eval"
args["epochs"] = 10

args["proportion_noise"] = 0.05


In [ ]:
import datasets
dataset = datasets.load_from_disk("./curricula/datasets/curriculum_10M_2024",num_proc=NUM_PROC_MAP)

In [ ]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("FacebookAI/roberta-base", max_len=512)
def replace_with_noise(example):
    tokenized = tokenizer(example["text"], return_special_tokens_mask=True, truncation=True, max_length=512)["input_ids"]
    example["text"] = tokenizer.decode(torch.randint(0, tokenizer.vocab_size, (len(tokenized),)), skip_special_tokens=True)
    example["source"] = "noise"
    return example
replace_with_noise(dataset[1000000])


In [ ]:
import torch 
torch.manual_seed(0)
ind_to_replace_with_noise = torch.randperm(int(len(dataset)*args["proportion_noise"]))
dl = dataset.to_list()
for i in ind_to_replace_with_noise:
    dl[i] = replace_with_noise(dl[i])
from collections import ChainMap


dataset = datasets.Dataset.from_dict({k: [line[k] for line in dl] for k in dl[0]})
dataset = dataset.shuffle(seed=42)
dataset.save_to_disk(args["dataset_folder"],num_proc=NUM_PROC_MAP)
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, args["curriculum_path"])
with open("./configs/random_10M_2024_noisy", 'w') as f:
    json.dump(args, f)
args

In [ ]:
ind_to_replace_with_noise[0]

## Test random

In [ ]:
args = {
    "dataset_folder": "./curricula/datasets/test",   
}
args["raw_dataset_folder_babylm"] = "./train_10M"
args["curriculum_path"] = "./curricula/test_random_curriculum"
args["eval_dataset_folder"] = "./curricula/datasets/test_eval"
args["epochs"] = 3

In [ ]:
import datasets
import torch 
torch.manual_seed(0)
import json
#dataset = load_dataset("text", data_dir ="/data/loriss21dm/babylm/train_test", cache_dir='/data/loriss21dm/cache')["train"] 
dataset = datasets.load_from_disk("./curricula/datasets/curriculum_10M_2024",num_proc=NUM_PROC_MAP,download_mode="force_redownload") # reuse one created above
dataset = dataset.shuffle()
dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//1000]) # TODO
dataset.save_to_disk(args["dataset_folder"],num_proc=NUM_PROC_MAP)

random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, args["curriculum_path"])


dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//10]) # TODO
dataset.save_to_disk(args["eval_dataset_folder"],num_proc=NUM_PROC_MAP)

with open("./configs/test", 'w') as f:
    json.dump(args, f)
args

## New Curriculum based on Influence during learning on randomly shuffled data

In [ ]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.stats import f
from scipy.stats import lognorm


import json
import os
import pandas as pd
import torch
import datasets
from datasets import load_dataset



def gaussian_filter(size, **args):
    indices = np.arange(-size, size+1, 1)
    weights =  norm.pdf(indices, **args)
    return weights# / np.sum(weights)


def f_filter(size, **args):
    indices = np.arange(-size, size+1, 1)
    print(indices)
    weights = f.pdf(indices+1, **args)
    return weights# / np.sum(weights)
def lognorm_filter(size, **args):
    #print(args)
    indices = np.arange(-size, size+1, 1)
    #print(indices)
    weights = lognorm.pdf(indices+1, **args)
    return weights / np.sum(weights)



with open("./configs/random_10M_2024") as f:
    config = json.load(f)
    print(config)
config["influence_output_dir"] = os.path.join("./influence", os.path.basename(config["curriculum_path"]))
dataset = datasets.load_from_disk(config["dataset_folder"],num_proc=NUM_PROC_MAP)
df = pd.DataFrame({int(result_checkpoint): torch.load(os.path.join(config["influence_output_dir"],result_checkpoint),weights_only=True,map_location="cpu").numpy().flatten() for result_checkpoint in os.listdir(config["influence_output_dir"])})
df.sort_index(axis=1)

df = df.reindex(sorted(df.columns, reverse=False), axis=1)
influence_cols = df.columns
df["total"] = df.sum(axis=1)
df[["text", "source"]] = dataset.to_pandas()
df["document_lenght"] = df["text"].str.split(r"\S+").str.len()




def reweight_df(df, influence_cols, filter_weights):
    scores = pd.DataFrame(np.apply_along_axis(lambda m: np.convolve(m,filter_weights, mode="valid")[1:-1], axis=1, arr=df[influence_cols].to_numpy()))
    scores.columns = influence_cols
    df_reweighted = df.copy()
    df_reweighted[influence_cols] = scores
    df_reweighted["total"] = df_reweighted[influence_cols].sum(axis=1)
    return df_reweighted



filter_weights = lognorm_filter(len(influence_cols), s=1, loc=0, scale=0.5)
plt.plot(np.arange(-len(influence_cols),len(influence_cols)+1, 1),filter_weights)
plt.vlines([0], ymin=0, ymax=max(filter_weights)+0.1, colors=["red"])
df_reweighted = reweight_df(df, influence_cols, lognorm_filter(len(influence_cols), s=1, loc=0, scale=0.5))
display(df_reweighted)




In [ ]:
dataset

In [ ]:
len(packed_dataset[0]["input_ids"])

In [ ]:
args = {
    "dataset_folder": config["dataset_folder"],   
}
args["raw_dataset_folder_babylm"] = "./train_10M"
args["curriculum_path"] = "./curricula/lognorm_curriculum_from_random_10M_2024"
args["eval_dataset_folder"] = config["eval_dataset_folder"]
args["epochs"] = config["epochs"]

torch.save(torch.stack([torch.tensor(df[checkpoint].sort_values(ascending=False).index.to_numpy()) for checkpoint in influence_cols]), args["curriculum_path"])


with open("./configs/lognorm_curriculum_from_random_10M_2024", 'w') as f:
    json.dump(args, f)